<a href="https://colab.research.google.com/github/ZakkiMubarrok/ubiquitos/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# !pip install Pillow
# !pip install torch torchvision
# !pip install git+https://github.com/facebookresearch/detectron2.git


import os
import random
import json
from sklearn.model_selection import train_test_split
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from PIL import Image

In [17]:
path = r'/content/dataset/mentah'

for file in os.listdir(path):
    if not file.endswith(".jpg"):
        img = Image.open(f"{path}/{file}")
        file_name, file_ext = os.path.splitext(file)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(f'/content/dataset/photos-unrized/{file_name}.jpg'.format(file_name))
    else :
        img = Image.open(f"{path}/{file}")
        file_name, file_ext = os.path.splitext(file)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(f'/content/dataset/photos-unrized/{file_name}.jpg'.format(file_name))

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [18]:
path = r'/content/dataset/photos-unrized'
output_directory_resized = r'/content/dataset/photos'

# Pastikan direktori output ada, jika tidak, buat
if not os.path.exists(output_directory_resized):
    os.makedirs(output_directory_resized)

# Tentukan ukuran target (misalnya: 640x480)
target_size = (640, 480)

for file in os.listdir(path):
    if file.endswith(".jpg"):
        img = Image.open(os.path.join(path, file))
        file_name, _ = os.path.splitext(file)

        # Ubah ukuran gambar
        resized_img = img.resize(target_size)

        # Simpan gambar yang sudah diubah ukurannya
        resized_img.save(os.path.join(output_directory_resized, f"{file_name}.jpg"))

In [19]:
# Fungsi untuk membagi dataset menjadi train dan validation secara acak
def split_dataset(annotation_file, train_ratio=0.8):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # Memecah data menjadi train dan val
    train_data, val_data = train_test_split(data["annotations"], train_size=train_ratio, random_state=42)

    # Membuat file annotation untuk train dan val
    train_data_json = {"info": data["info"], "licenses": data["licenses"], "images": data["images"], "annotations": train_data, "categories": data["categories"]}
    val_data_json = {"info": data["info"], "licenses": data["licenses"], "images": data["images"], "annotations": val_data, "categories": data["categories"]}

    train_annotation_file = os.path.splitext(annotation_file)[0] + "_train.json"
    val_annotation_file = os.path.splitext(annotation_file)[0] + "_val.json"

    with open(train_annotation_file, 'w') as train_file:
        json.dump(train_data_json, train_file, indent=2)

    with open(val_annotation_file, 'w') as val_file:
        json.dump(val_data_json, val_file, indent=2)

    return train_annotation_file, val_annotation_file

# Direktori root yang berisi file annotation
root_directory = "/content/dataset"

# Iterasi melalui setiap file annotation di direktori root
for annotation_file in os.listdir(root_directory):
    if annotation_file.endswith(".json"):
        annotation_file_path = os.path.join(root_directory, annotation_file)

        # Membagi dataset menjadi train dan validation secara acak
        train_annotation_file, val_annotation_file = split_dataset(annotation_file_path)

        # Registrasi dataset COCO untuk train dan validation
        register_coco_instances(f"custom_dataset_train_{annotation_file}", {}, train_annotation_file, os.path.join(root_directory, "photos"))
        register_coco_instances(f"custom_dataset_val_{annotation_file}", {}, val_annotation_file, os.path.join(root_directory, "photos"))



AssertionError: ignored

In [20]:
# Iterasi melalui setiap file annotation di direktori root untuk melatih model
for annotation_file in os.listdir(root_directory):
    if annotation_file.endswith("_train.json"):
        annotation_file_path = os.path.join(root_directory, annotation_file)

        # Registrasi dataset COCO untuk train dan validation
        dataset_name = f"custom_dataset_train_{annotation_file}"  # Sesuaikan dengan nama yang digunakan pada registrasi
        # register_coco_instances(dataset_name, {}, annotation_file_path, os.path.join(root_directory, "photos"))

        # Inisialisasi metadata untuk dataset
        metadata = MetadataCatalog.get(dataset_name)  # Menggunakan nama dataset yang sesuai

        # Konfigurasi model Detectron2
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
        cfg.DATASETS.TRAIN = (dataset_name,)  # Menggunakan nama dataset yang sesuai
        cfg.DATASETS.TEST = ()  # Sesuaikan dengan dataset pengujian jika diperlukan
        cfg.DATALOADER.NUM_WORKERS = 2
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Menggunakan pre-trained weights
        cfg.SOLVER.IMS_PER_BATCH = 2
        cfg.SOLVER.BASE_LR = 0.00025
        cfg.SOLVER.MAX_ITER = 5000
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

        # Melatih model
        trainer = DefaultTrainer(cfg)
        trainer.resume_or_load(resume=False)
        trainer.train()


[01/02 15:07:34 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[01/02 15:07:35 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/02 15:07:47 d2.utils.events]:  eta: 0:30:32  iter: 19  total_loss: 1.776  loss_cls: 1.441  loss_box_reg: 0.3051  loss_rpn_cls: 0.0208  loss_rpn_loc: 0.01338    time: 0.3762  last_time: 0.3428  data_time: 0.0245  last_data_time: 0.0052   lr: 4.9953e-06  max_mem: 3022M
[01/02 15:08:27 d2.utils.events]:  eta: 0:30:51  iter: 39  total_loss: 1.66  loss_cls: 1.329  loss_box_reg: 0.2711  loss_rpn_cls: 0.0237  loss_rpn_loc: 0.01693    time: 0.3816  last_time: 0.3641  data_time: 0.0111  last_data_time: 0.0058   lr: 9.9902e-06  max_mem: 3022M
[01/02 15:08:35 d2.utils.events]:  eta: 0:30:33  iter: 59  total_loss: 1.504  loss_cls: 1.092  loss_box_reg: 0.2665  loss_rpn_cls: 0.02043  loss_rpn_loc: 0.0115    time: 0.3766  last_time: 0.3615  data_time: 0.0091  last_data_time: 0.0115   lr: 1.4985e-05  max_mem: 3022M
[01/02 15:08:42 d2.utils.events]:  eta: 0:30:30  iter: 79  total_loss: 1.218  loss_cls: 0.8904  loss_box_reg: 0.2646  loss_rpn_cls: 0.02324  loss_rpn_loc: 0.01421    time: 0.3777  last_

In [34]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog

# Inisialisasi konfigurasi model
cfg = get_cfg()
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Gantilah dengan path ke file model yang telah Anda simpan
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Threshold skor untuk deteksi
cfg.DATASETS.TEST = ("custom_dataset_val",)  # Gantilah dengan dataset validasi Anda

# Inisialisasi predictor
predictor = DefaultPredictor(cfg)

# Path gambar yang akan diuji
image = cv2.imread("/content/dataset/photos/zebracross83.jpg")   # Gantilah dengan path ke gambar yang akan diuji

# Melakukan prediksi
outputs = predictor(image)

# Tampilkan hasil prediksi
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
# ... tambahkan informasi lain yang ingin Anda lihat

# Mendapatkan metadata (kategori, warna, dsb.)
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])
print("Labels:", metadata.thing_classes)


[01/02 17:03:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...


backbone.res2.0.conv1.norm.{bias, weight}
backbone.res2.0.conv1.weight
backbone.res2.0.conv2.norm.{bias, weight}
backbone.res2.0.conv2.weight
backbone.res2.0.conv3.norm.{bias, weight}
backbone.res2.0.conv3.weight
backbone.res2.0.shortcut.norm.{bias, weight}
backbone.res2.0.shortcut.weight
backbone.res2.1.conv1.norm.{bias, weight}
backbone.res2.1.conv1.weight
backbone.res2.1.conv2.norm.{bias, weight}
backbone.res2.1.conv2.weight
backbone.res2.1.conv3.norm.{bias, weight}
backbone.res2.1.conv3.weight
backbone.res2.2.conv1.norm.{bias, weight}
backbone.res2.2.conv1.weight
backbone.res2.2.conv2.norm.{bias, weight}
backbone.res2.2.conv2.weight
backbone.res2.2.conv3.norm.{bias, weight}
backbone.res2.2.conv3.weight
backbone.res3.0.conv1.norm.{bias, weight}
backbone.res3.0.conv1.weight
backbone.res3.0.conv2.norm.{bias, weight}
backbone.res3.0.conv2.weight
backbone.res3.0.conv3.norm.{bias, weight}
backbone.res3.0.conv3.weight
backbone.res3.0.shortcut.norm.{bias, weight}
backbone.res3.0.shortcut.w

tensor([], device='cuda:0', dtype=torch.int64)
Boxes(tensor([], device='cuda:0', size=(0, 4)))


AttributeError: ignored